<a href="https://colab.research.google.com/github/jellyho/DACON/blob/master/SNP_20221228.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns

data_path = '/content/drive/MyDrive/DACON/SNP/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
info = pd.read_csv(data_path + 'snp_info.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [9]:
train

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,...,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,0,0,0,2,A G,A G,A A,G A,C C,...,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,TRAIN_258,0,0,0,2,G G,A A,C A,A A,A A,...,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,TRAIN_259,0,0,0,1,A G,G G,A A,G A,A A,...,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,TRAIN_260,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G A,A G,A G,G A,G G,C A,G G,A


일단 염기서열을 SNP_01_01, 02로 가서 각각을 one_hot encoding으로 조져 그러면 각 그거당 4개의 열을 사용하면 됨.

In [10]:
info

,SNP_id,name,chrom,cm,pos
0,SNP_01,BTA-19852-no-rs,2,67.05460,42986890
1,SNP_02,ARS-USMARC-Parent-DQ647190-rs29013632,6,31.15670,13897068
2,SNP_03,ARS-BFGL-NGS-117009,6,68.28920,44649549
3,SNP_04,ARS-BFGL-NGS-60567,6,77.87490,53826064
4,SNP_05,BovineHD0600017032,6,80.50150,61779512
5,SNP_06,BovineHD0600017424,6,80.59540,63048481
6,SNP_07,Hapmap49442-BTA-111073,6,80.78000,64037334
7,SNP_08,BovineHD0600018638,6,82.68560,67510588
8,SNP_09,ARS-BFGL-NGS-37727,6,86.87400,73092782
9,SNP_10,BTB-01558306,7,62.06920,40827112


In [11]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

sc = StandardScaler()
mc = MinMaxScaler()

data = info[['chrom', 'cm', 'pos']]
data = sc.fit_transform(data)
pca = PCA(n_components=1)
info_factor = pca.fit_transform(data)
info_factor = mc.fit_transform(info_factor)

In [12]:
def pre_processing(df, isTrain=True):
  df = df.drop(columns=['father', 'mother', 'gender', 'id'])
  dic0 = {'A':0, 'T':0, 'G':1, 'C':1}
  dic1 = {'A':0, 'T':1, 'G':0, 'C':1}
  dicClass = {'A':0, 'B':1, 'C':2}
  dictrait = {1:0, 2:1}
  for i in range(1, 16):
    l0 = []
    l1 = []
    l2 = []
    l3 = []
    value = df[f"SNP_{'0'if i < 10 else ''}{i}"].values
    for v in value:
      s = v.split(' ')
      l0.append(dic0[s[0]])
      l1.append(dic1[s[0]])
      l2.append(dic0[s[1]])
      l3.append(dic1[s[1]])
    df[f"SNP_{'0'if i < 10 else ''}{i}_0"] = l0
    df[f"SNP_{'0'if i < 10 else ''}{i}_1"] = l1
    df[f"SNP_{'0'if i < 10 else ''}{i}_2"] = l2
    df[f"SNP_{'0'if i < 10 else ''}{i}_3"] = l3
    df[f"Mul_SNP_{'0'if i < 10 else ''}{i}_0"] = l0 * info_factor[i-1]
    df[f"Mul_SNP_{'0'if i < 10 else ''}{i}_1"] = l1 * info_factor[i-1]
    df[f"Mul_SNP_{'0'if i < 10 else ''}{i}_2"] = l2 * info_factor[i-1]
    df[f"Mul_SNP_{'0'if i < 10 else ''}{i}_3"] = l3 * info_factor[i-1]

  df = df.drop(columns=[f"SNP_{'0'if j < 10 else ''}{j}" for j in range(1, 16)])
  df['trait'] = df['trait'].apply(lambda x: dictrait[x])
  if isTrain:
    df['class'] = df['class'].apply(lambda x: dicClass[x])
    return df.drop(columns=['class']), df['class'].values
  else:
    return df

In [13]:
train_X, train_y = pre_processing(train)
test_X = pre_processing(test, False)

<ipython-input-12-db077193c942>:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"SNP_{'0'if i < 10 else ''}{i}_2"] = l2
<ipython-input-12-db077193c942>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"SNP_{'0'if i < 10 else ''}{i}_3"] = l3
<ipython-input-12-db077193c942>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented fra

In [14]:
import lightgbm as lgbm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

kfold = StratifiedKFold(n_splits=50).split(train_X, train_y)

lgbm_models = []
lgbm_scores = []

for (ktrain, kvalid) in kfold:
  dtrain = lgbm.Dataset(train_X.iloc[ktrain, :], train_y[ktrain])
  dvalid = lgbm.Dataset(train_X.iloc[kvalid, :], train_y[kvalid])
  params = {'objective': 'multiclass', 'num_class':3, 'force_col_wise': True, 'bagging_freq': 1, 'learning_rate': 0.001, 'random_state':413}
  model= lgbm.train(params=params, train_set=dtrain, num_boost_round=100000, valid_sets=dvalid, early_stopping_rounds=2000, verbose_eval=1000)
  pred = model.predict(train_X.iloc[kvalid, :])
  score = log_loss(train_y[kvalid], pred)
  if score < 1e-5:
    lgbm_scores.append(score)
    lgbm_models.append(model)

Training until validation scores don't improve for 2000 rounds.
[1000]	valid_0's multi_logloss: 0.497145
[2000]	valid_0's multi_logloss: 0.266403
[3000]	valid_0's multi_logloss: 0.14306
[4000]	valid_0's multi_logloss: 0.0772544
[5000]	valid_0's multi_logloss: 0.0461213
[6000]	valid_0's multi_logloss: 0.0285288
[7000]	valid_0's multi_logloss: 0.0150809
[8000]	valid_0's multi_logloss: 0.00889926
[9000]	valid_0's multi_logloss: 0.00605378
[10000]	valid_0's multi_logloss: 0.00458639
[11000]	valid_0's multi_logloss: 0.00340182
[12000]	valid_0's multi_logloss: 0.00282819
[13000]	valid_0's multi_logloss: 0.00233249
[14000]	valid_0's multi_logloss: 0.00169267
[15000]	valid_0's multi_logloss: 0.0011542
[16000]	valid_0's multi_logloss: 0.000889434
[17000]	valid_0's multi_logloss: 0.000757127
[18000]	valid_0's multi_logloss: 0.000582713
[19000]	valid_0's multi_logloss: 0.000418628
[20000]	valid_0's multi_logloss: 0.000297102
[21000]	valid_0's multi_logloss: 0.00022067
[22000]	valid_0's multi_logl

베이지안 최적화 돌려서 일단 어느정도 성능 나오도록 하자.

In [ ]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

kfold = StratifiedKFold(n_splits=50).split(train_X, train_y)

xgb_models = []
xgb_scores = []

for (ktrain, kvalid) in kfold:
  model = xgb.XGBClassifier(objective='multi:softmax', n_estimators=30000,  num_class=3, learning_rate=0.05, random_state=413, eval_metric=['mlogloss'])
  model.fit(train_X.iloc[ktrain, :], train_y[ktrain], verbose=1000, eval_set=[(train_X.iloc[kvalid, :], train_y[kvalid])], early_stopping_rounds=1000)
  iter = model.best_iteration
  pred = model.predict_proba(train_X.iloc[kvalid, :])
  score = log_loss(train_y[kvalid], pred)
  xgb_scores.append(score)
  xgb_models.append(model)

In [30]:
preds = []
for idx in np.argsort(lgbm_scores)[:1]:
  pred = lgbm_models[idx].predict(test_X)
  preds.append(pred)
final_pred_lgbm = np.sum(preds, axis=0)
preds = []
for idx in np.argsort(xgb_scores)[:1]:
  pred = xgb_models[idx].predict_proba(test_X)
  preds.append(pred)
final_pred_xgb = np.sum(preds, axis=0)

In [31]:
final_pred = final_pred_lgbm + final_pred_xgb

In [32]:
submission['class'] = np.argmax(final_pred, axis=1)
dicClassRev = {0: 'A', 1: 'B', 2: 'C'}
submission['class'] = submission['class'].apply(lambda x: dicClassRev[x])
submission.to_csv('submission_lgbm_xgb.csv', index=False)

일단 이 전처리로 성능 향상을 최대로 해봅시다.
crossfold방식으로 5개 앙상블
다른 모델과 앙상블(xgboost)